### Парсинг файлов после MAST 

In [1]:
from bs4 import BeautifulSoup

In [3]:
with open("mast_interpro_1_cl.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

In [4]:
motifs = soup.motifs.findAll('motif')
length = [motif['length'] for motif in motifs]
length

['35', '31', '29']

На 24 месте в первом мотиве не должен стоять Cys. Проверим это и запишем проверенные сиквенсы в отдельные файл.

In [5]:
sequences = soup.sequences.findAll('sequence')

In [6]:
import re
reg = re.compile('[^a-zA-Z ]')

In [ ]:
Cys_seq_names = set()
Cys_seq = set()
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True:
            if reg.sub('', seg.data.string)[int(seg.find(idx = '0')['pos']) - int(seg['start']) + 23] != 'C':
                Cys_seq_names.add(seq['name'])
                Cys_seq.add(seq)

In [ ]:
with open('Cysless_sequences_4.xml', 'w') as f:
    for i in Cys_seq:
        f.write(str(i) + '\n')

Отдельно запишем данные в формате fasta

In [ ]:
with open('protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [ ]:
new_file = []
for name in Cys_seq_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [ ]:
with open('Cysless_proteins_4.fasta', 'w') as f:
    f.write(''.join(new_file))

### Парсинг результатов pfam

In [23]:
import pandas as pd

Переведем txt таблицу в удобный нам пандас.

In [62]:
columns = []
d = []
with open('pfam_4_cl.txt', 'r') as f:
    for line in f:
        if line.startswith('<'):
            columns = line.split('> <')
        else:
            d.append(line.split())
columns[0] = columns[0].strip('<')
columns[-1] = columns[-1].strip('>\n')

In [63]:
data = pd.DataFrame(d, columns=columns)
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,M1FCN8,32,128,22,129,PF13426.7,PAS_9,Domain,11,103,104,76.6,1.5e-21,1,CL0183
1,M1FCN8,148,251,148,251,PF13426.7,PAS_9,Domain,1,104,104,64.4,9.1e-18,1,CL0183
2,M1FCN8,265,416,263,416,PF00990.21,GGDEF,Domain,3,161,161,125.5,1.5e-36,1,CL0276
3,V4Y023,135,236,134,236,PF13426.7,PAS_9,Domain,2,104,104,42.9,4.7e-11,1,CL0183
4,V4Y023,257,361,255,361,PF13426.7,PAS_9,Domain,3,104,104,76.0,2.3e-21,1,CL0183


### Идентифекация LOV 

Отбор по отсутствию Cys уже прошел, поэтому необходимо отсеять по p-value. 

In [116]:
with open("Cysless_sequences_4.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

Отобрали dictionary нужных наименований со списком позиций трех мотивов (этих троек мб несколько в последовательности)

In [117]:
sequences = soup.sequences.find_all('sequence')
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if (bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True and bool(seg.find(idx = '2')) == True):
            hits = seg.find_all('hit')
            if (float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15 and float(hits[2]['pvalue']) <= 1e-15):
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos']), 
                                                         int(hits[2]['pos'])])

In [118]:
new_names = []
wrong = []
for k in seqs.keys():
    seq_data = data.loc[data['seq id'] == k] # data - таблица с результатами pfam, здесь отсев по id
    df = seq_data.loc[seq_data['clan'] == 'CL0183'] # здесь отсев по id клана (группы) домена PAS
    b = False
    for hit in seqs[k]:
        for i in range(df.shape[0]):
            if int(df.iloc[i, 3]) - hit[0] <= 2 and hit[2] + 28 - int(df.iloc[i, 4]) <= 10:
                b = True
    if not b:
        data.drop(data[data['seq id'] == k].index, inplace=True)
        wrong.append(k)
    else:
        new_names.append(k)   

In [119]:
with open('/home/dashik/projects/181015_conf_mipt_18/protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [120]:
new_file = []
for name in new_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [121]:
with open('final_proteins_4_cl.fasta', 'w') as f:
    f.write(''.join(new_file))

Проверим на пересечения

In [122]:
with open('/home/dashik/projects/181015_conf_mipt_18/With_Cys/final_proteins_4.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [123]:
with open('final_proteins_4_cl.fasta', 'r') as f:
    fasta_cl = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta_cl.setdefault(name, []).append(line)

In [124]:
repeat = list(fasta.keys() & fasta_cl.keys())
len(repeat)

77

In [125]:
for r in repeat:
    fasta_cl.pop(r)

In [126]:
new_file = []
for name in fasta_cl.keys():
    new_file.append(name + '\n')
    for s in fasta_cl.get(name):
        new_file.append(s + '\n')
with open('final_proteins_4_cl.fasta', 'w') as f:
    f.write(''.join(new_file))

Удалили повторы и объеденили финальные последоватеьнсти в оди файл, для построения дерева и выравнивания белковых последовательностей.

In [1]:
 # data - df pfam
'''Доступ к Uniprot'''
from Bio import ExPASy
from Bio import SwissProt
from tqdm import trange

In [2]:
def kingdom(s):
    handle = ExPASy.get_sprot_raw(s[0])
    try:
        record = SwissProt.read(handle)
        return record.organism_classification[0] + ' ' + record.organism_classification[1]
    except ValueError:
        return None

In [64]:
data['kingdom'] = None

In [65]:
with open('final_proteins_4_cl.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)        

In [66]:
len(fasta)

69

In [67]:
new_data = pd.DataFrame()
for k in fasta.keys():
    new_data = pd.concat([new_data, data.loc[data['seq id'] == k, :]], ignore_index=True)

In [69]:
for i in trange(400):
    new_data.iloc[i, 15] = kingdom(new_data.iloc[i,:])

100%|██████████| 400/400 [05:46<00:00,  1.12it/s]


In [68]:
new_data.shape

(400, 16)

In [70]:
tfile = open('domains_4_cl.csv', 'w')
tfile.write(new_data.to_csv())
tfile.close()

In [61]:
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan,kingdom
0,A0A2T6DB03,7,113,7,117,PF00072.24,Response_reg,Domain,1,108,112,56.3,3.1e-15,1,CL0304,None
1,A0A2T6DB03,165,249,164,249,PF08447.12,PAS_3,Domain,2,89,89,58.6,5.5e-16,1,CL0183,None
2,A0A2T6DB03,275,381,273,381,PF13426.7,PAS_9,Domain,3,104,104,67.8,8e-19,1,CL0183,None
3,A0A2T6DB03,526,631,524,632,PF08448.10,PAS_4,Domain,3,109,110,34.7,1.6e-08,1,CL0183,None
4,A0A2T6DB03,647,706,645,710,PF00512.25,HisKA,Domain,3,63,67,29.4,5.8e-07,1,CL0025,None


#### Находим LOV

In [1]:
from bs4 import BeautifulSoup
with open("mast_known.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

sequences = soup.sequences.findAll('sequence')

In [5]:
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True:
            hits = seg.find_all('hit')
            if float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15:
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos'])]) 
                                                        

In [18]:
from Bio import SeqIO
fasta = {}
for rec in SeqIO.parse('Known_proteins.fasta', 'fasta'):
    fasta[rec.id] = str(rec.seq)

In [21]:
seqs

{'AAC05083.1': [[134, 179], [411, 456]],
 'sp|O48963.1|PHOT1_ARATH': [[195, 240], [473, 518]],
 'sp|Q00TW1|Q00TW1_OSTTA': [[27, 72], [202, 247]],
 'EDP03413.1': [[18, 63], [211, 256]],
 'AGG62438.1': [[23, 68]],
 'sp|Q8PJH6|Q8PJH6_XANAC': [[37, 82]],
 'BAA36192.2': [[927, 972]],
 'BAF91488.1': [[215, 260]],
 'AAL53921.1': [[30, 75]],
 'sp|P58724.1|PHOT_LISMO': [[17, 62]],
 'sp|Q94BT6.2|ADO1_ARATH': [[43, 97]],
 'CAL55375.1': [[108, 153]],
 'EAA28370.3': [[69, 125]],
 'AAF32298.2': [[52, 106]],
 'WP_011242514.1': [[379, 424]],
 'pdb|3SW1|A': [[34, 79]],
 'CAA63964.2': [[389, 445]],
 'sp|Q31RU9|Q31RU9_SYNP7': [[34, 79]],
 'pdb|3P7N|A': [[72, 117]],
 'pdb|4HNB|A': [[17, 62]]}

In [20]:
new_seq = {}
for key in seqs.keys():
    if key in fasta:
        new_seq[key] = seqs[key]
seqs = new_seq

In [37]:
data = pd.read_csv('/home/dashik/projects/190115_cysless_lov/known_pfam.csv')

In [38]:
def IfLOV(row):
    if row['clan'] == 'CL0183':
        for hit in seqs[row['seq id']]:
            if (int(row[3]) - hit[0]) <=5 and (hit[1] + 44 - int(row[4])) <= 10:
                return 'LOV'
            else: 
                return row['hmm name']
    else:
        return row['hmm name']

In [43]:
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,EDP03413.1,29,124,21,125,PF13426.7,LOV,Domain,8,103,104,78.9,3.000000e-22,1,CL0183
1,EDP03413.1,222,318,214,318,PF13426.7,PAS_9,Domain,8,104,104,77.8,6.200000e-22,1,CL0183
2,EDP03413.1,405,713,404,713,PF00069.25,Pkinase,Domain,2,264,264,203.6,3.400000e-60,1,CL0016
3,BAF91488.1,121,168,119,170,PF07716.15,bZIP_2,Family,3,49,54,31.5,1.400000e-07,1,CL0018
4,BAF91488.1,226,318,217,322,PF13426.7,LOV,Domain,8,100,104,62.9,2.700000e-17,1,CL0183


In [40]:
data['hmm name'] = data.apply(IfLOV, axis = 1)

In [45]:
tfile = open('known_domains.csv', 'w')
tfile.write(data.to_csv())
tfile.close()

#### Находим TLOV 

In [194]:
from tqdm import tqdm

for name in tqdm(seqs.keys()):
    indexes = data[(data['seq id'] == name) & (data['hmm name'] == 'LOV')].index.values
    for i in indexes[:-1]:
        if (data.loc[i+1, 'hmm name'] == 'LOV' and 
            int(data.loc[i+1, 'envelope start']) - int(data.loc[i, 'envelope end']) <= 125):
            data.loc[i, 'envelope end'] = data.loc[i+1, 'envelope end']
            data.loc[i, 'hmm name'], data.loc[i+1, 'hmm name'] = 'TLOV', '_LOV'

100%|██████████| 69/69 [00:00<00:00, 363.84it/s]


In [84]:
data = data.loc[data['hmm name'] != '_LOV']

In [195]:
tfile = open('domains_LOV_4_cl.csv', 'w')
tfile.write(data.to_csv())
tfile.close()

## Данные для выравнивания

Данные для выравнивания будут выравниваться только для участка содержащего LOV.
В случае TLOV его разбили на иначальные два лова (TLOV есть только в цистеиновых, так как он встречает только у высших растений) Параллельно будет создана таблица для окрашивания дерева, по наличию цистеина и для двух половинок цитеина (интересно как они будут распологаться)

In [46]:
data = pd.read_csv('/home/dashik/projects/190115_cysless_lov/known_domains.csv')

In [47]:
data.drop('Unnamed: 0', axis = 1, inplace = True)
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,EDP03413.1,29,124,21,125,PF13426.7,LOV,Domain,8,103,104,78.9,3.000000e-22,1,CL0183
1,EDP03413.1,222,318,214,318,PF13426.7,PAS_9,Domain,8,104,104,77.8,6.200000e-22,1,CL0183
2,EDP03413.1,405,713,404,713,PF00069.25,Pkinase,Domain,2,264,264,203.6,3.400000e-60,1,CL0016
3,BAF91488.1,121,168,119,170,PF07716.15,bZIP_2,Family,3,49,54,31.5,1.400000e-07,1,CL0018
4,BAF91488.1,226,318,217,322,PF13426.7,LOV,Domain,8,100,104,62.9,2.700000e-17,1,CL0183


In [48]:
data_LOV = data[data['hmm name'] == 'LOV'].reset_index(drop = True)
#data_TLOV = data[data['hmm name'] == 'TLOV'].reset_index(drop = True)
#data_LOV_ = data[data['hmm name'] == '_LOV'].reset_index(drop = True)

In [49]:
from Bio import SeqIO
fasta = {}
for rec in SeqIO.parse('Known_proteins.fasta', 'fasta'):
    fasta[rec.id] = str(rec.seq)

In [212]:
with open('/home/dashik/projects/190115_cysless_lov/With_Cys/final_proteins_4.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)

In [50]:
new_fasta = {}
for k,v in fasta.items():
    if k in list(data_LOV['seq id']):
        for n in range(data_LOV.loc[data_LOV['seq id'] == k, 'envelope start'].shape[0]): 
            i, j = (int(data_LOV.loc[data_LOV['seq id'] == k, 'envelope start'].reset_index(drop = True)[n]), 
                    int(data_LOV.loc[data_LOV['seq id'] == k, 'envelope end'].reset_index(drop = True)[n]))
            LOV = ''.join(v)[i-1: j]
            new_fasta['>' + k + str(n)] = LOV
'''for k,v in fasta.items():
    if k in list(data_TLOV['seq id']):
        for n in range(data_TLOV.loc[data_TLOV['seq id'] == k, 'envelope start'].shape[0]): 
            i, j = (int(data_TLOV.loc[data_TLOV['seq id'] == k, 'envelope start'].reset_index(drop = True)[n]), 
                    int(data_TLOV.loc[data_TLOV['seq id'] == k, 'envelope end'].reset_index(drop = True)[n]))
            LOV = ''.join(v)[i-1: j]
            new_fasta['>' + k + str(n) + '_T1'] = LOV
for k,v in fasta.items():
    if k in list(data_LOV_['seq id']):
        for n in range(data_LOV_.loc[data_LOV_['seq id'] == k, 'envelope start'].shape[0]): 
            i, j = (int(data_LOV_.loc[data_LOV_['seq id'] == k, 'envelope start'].reset_index(drop = True)[n]), 
                    int(data_LOV_.loc[data_LOV_['seq id'] == k, 'envelope end'].reset_index(drop = True)[n]))
            LOV = ''.join(v)[i-1: j]
            new_fasta['>' + k + str(n) + '_T2'] = LOV

  '''  

"for k,v in fasta.items():\n    if k in list(data_TLOV['seq id']):\n        for n in range(data_TLOV.loc[data_TLOV['seq id'] == k, 'envelope start'].shape[0]): \n            i, j = (int(data_TLOV.loc[data_TLOV['seq id'] == k, 'envelope start'].reset_index(drop = True)[n]), \n                    int(data_TLOV.loc[data_TLOV['seq id'] == k, 'envelope end'].reset_index(drop = True)[n]))\n            LOV = ''.join(v)[i-1: j]\n            new_fasta['>' + k + str(n) + '_T1'] = LOV\nfor k,v in fasta.items():\n    if k in list(data_LOV_['seq id']):\n        for n in range(data_LOV_.loc[data_LOV_['seq id'] == k, 'envelope start'].shape[0]): \n            i, j = (int(data_LOV_.loc[data_LOV_['seq id'] == k, 'envelope start'].reset_index(drop = True)[n]), \n                    int(data_LOV_.loc[data_LOV_['seq id'] == k, 'envelope end'].reset_index(drop = True)[n]))\n            LOV = ''.join(v)[i-1: j]\n            new_fasta['>' + k + str(n) + '_T2'] = LOV\n\n  "

In [214]:
new_file = []
for k, v in new_fasta.items():
    new_file.append(k + '\n')
    new_file.append(v + '\n')

In [215]:
with open('LOVs.fasta', 'a') as f:
    f.write(''.join(new_file))

Чтение для определения количества ловов в выравнивании до отбора

In [3]:
with open('LOVs.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)

In [4]:
len(fasta)

12646

Для перво попытки было решено взять 10% от всей выборки с Сys. В каждом царстве отбор идет отдельно. 

In [51]:
new_fasta

{'>EDP03413.10': 'TFVVADATLPDCPLVYASEGFYAMTGYGPDEVLGHNCRFLQGEGTDPKEVQKIRDAIKKGEACSVRLLNYRKDGTPFWNLLTVTPIKTPDGRVSKFVGVQVDVTS',
 '>BAF91488.10': 'QNFVITDASLPDNPIVYASRGFLTLTGYSLDQILGRNCRFLQGPETDPRAVDKIRNAITKGVDTSVCLLNYRQDGTTFWNLFFVAGLRDSKGNIVNYVGVQSKVSE',
 '>CAL55375.10': 'AEGITIADFSKPDQPLIYANIGFESMTGYSVQETLGKNCRFLQGPGTDLKELGKVRAAITKGEACTVVLKNYKKSGEEFMNQLSLTPIRDGEGNVMYYVGIQSDITE',
 '>AAF32298.20': 'PSFIVSDALEPDFPLIYVNRVFEVFTGYRADEVLGRNCRFLQYRDPRAQRRHPLVDPVVVSEIRRCLEEGIEFQGELLNFRKDGTPLVNRLRLAPIRDDDGTITHVIGIQV',
 '>sp|Q94BT6.2|ADO1_ARATH0': 'PCGFVVTDAVEPDQPIIYVNTVFEMVTGYRAEEVLGGNCRFLQCRGPFAKRRHPLVDSMVVSEIRKCIDEGIEFQGELLNFRKDGSPLMNRLRLTPIYGDDDTITHIIGIQFFI',
 '>sp|O48963.1|PHOT1_ARATH0': 'TFVVSDATKPDYPIMYASAGFFNMTGYTSKEVVGRNCRFLQGSGTDADELAKIRETLAAGNNYCGRILNYKKDGTSFWNLLTIAPIKDESGKVLKFIGMQVEVSK',
 '>AAC05083.10': 'TFVVSDASRPGHPIMYASAGFFNMTGYTSKEVVGRNCRFLQGSGTDPAEIAKIRQALANGSNYCGRVLNYKKDGTAFWNLLTIAPIKDEEGRVLKFIGMQVEVSK',
 '>BAA36192.20': 'KSFVITDPRLPDNPIIFASDRFLELTEYTREEVLGNNCRFLQGRGTDRKAVQLIRDA